In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os

# Lecture des fichiers 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_dir = '/content/drive/MyDrive/DATASET/dataset_ai4utg/Generation test sans pretty print/GenerationTest/'

# Creation des listes 

In [4]:
from tqdm import tqdm

In [5]:
bdd = []
sbt = []

for i  in tqdm (range(1000), desc="loading..."):
  path_iteration = data_dir+'/Dir_Fonction_'+str(i+1)
  bdd_path = path_iteration+"/BDD_Fonction_"+str(i+1)+".txt"
  stb_path = path_iteration+"/SBT_Generation_Output/Fonction_"+str(i+1)+"_SBT.txt"
  bdd_txt = open(bdd_path,'r').read()
  sbt_txt = open(stb_path,'r').read()
  bdd.append(bdd_txt)
  sbt.append(sbt_txt)

loading...: 100%|██████████| 1000/1000 [29:28<00:00,  1.77s/it]


# Transformation en Dataframe

In [6]:
df = pd.DataFrame(bdd, columns=['bdd'])

In [7]:
df['sbt'] = sbt

# Split train / test

In [8]:
train_bdd = df['bdd'].iloc[:95]
test_bdd = df['bdd'].iloc[-5:]

train_sbt = df['sbt'].iloc[:95]
test_sbt = df['sbt'].iloc[-5:]

# Tokenization

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [10]:
def tokenize(x):
  tokenizer = Tokenizer(filters='!$%&*,./;<>?@[\\]^`_{|}~\t\n')
  tokenizer.fit_on_texts(x)
  sequences = tokenizer.texts_to_sequences(x)
  return tokenizer, sequences

In [11]:
tokenizer_sbt, sequences_sbt = tokenize(df['sbt'])
tokenizer_bdd, sequences_bdd = tokenize(df['bdd'])

# Padding des sequences 

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
max_len_sequence_sbt = max([len(x) for x in sbt])
print('len max sbt :',220)
max_len_sequence_bdd = max([len(x) for x in bdd])
print('len max bdd :',50)

len max sbt : 220
len max bdd : 50


In [14]:
word = sbt[0].split(' ')
len(word)

219

In [15]:
tokenizer_bdd.word_index

{'the': 1,
 'fonction': 2,
 'check': 3,
 'result': 4,
 'true': 5,
 '=': 6,
 'i': 7,
 '1': 8,
 'feature:': 9,
 '#': 10,
 'to': 11,
 'scenario:': 12,
 'bdd': 13,
 'given': 14,
 'and': 15,
 'when': 16,
 'call': 17,
 'then': 18,
 'should': 19,
 'be': 20,
 'f': 21,
 'c': 22,
 'h': 23,
 'v': 24,
 'w': 25,
 'l': 26,
 'o': 27,
 'j': 28,
 'g': 29,
 'z': 30,
 'e': 31,
 'q': 32,
 'b': 33,
 'n': 34,
 'k': 35,
 'm': 36,
 'x': 37,
 'u': 38,
 't': 39,
 'r': 40,
 's': 41,
 'd': 42,
 'a': 43,
 'y': 44,
 'p': 45,
 '662': 46,
 '648': 47,
 '399': 48,
 '722': 49,
 '90': 50,
 '139': 51,
 '550': 52,
 '659': 53,
 '135': 54,
 '147': 55,
 '273': 56,
 '759': 57,
 '196': 58,
 '59': 59,
 '262': 60,
 '21': 61,
 '478': 62,
 '468': 63,
 '851': 64,
 '73': 65,
 '533': 66,
 '395': 67,
 '809': 68,
 '300': 69,
 '839': 70,
 '95': 71,
 '96': 72,
 '653': 73,
 '665': 74,
 '489': 75,
 '304': 76,
 '729': 77,
 '937': 78,
 '182': 79,
 '164': 80,
 '619': 81,
 '179': 82,
 '862': 83,
 '598': 84,
 '485': 85,
 '501': 86,
 '258': 87,
 

In [16]:
list_shape = []
for i in sbt:
  list_shape.append(i)


In [17]:
def padding(x,length):
  padded = pad_sequences(x,maxlen=length,padding='post',truncating='post')
  return padded

In [18]:
padded_sbt = padding(sequences_sbt,220)
padded_bdd = padding(sequences_bdd,220)
#padded_sbt = np.expand_dims(padded_sbt,axis=-1)
#padded_bdd = np.expand_dims(padded_bdd,axis=-1)


In [19]:
padded_bdd[0]

array([   9,    3,    1,    2,    8,   10,    3,    1,    4,   11,    5,
         12,   13,    2,    8,    8,   14,   33,    6, 1107,   15,   22,
          6,   57,   16,    7,   17,    2,    8,   18,    1,    4,   19,
         20,    5,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [20]:
print(padded_bdd.shape)
print(padded_sbt.shape)

(1000, 220)
(1000, 220)


In [21]:
print(len(tokenizer_bdd.word_index))
print(len(tokenizer_sbt.word_index))

1909
1052


# Conversion en one hot encoding

In [22]:
#sbt_token_index = dict([(char, i) for i, char in enumerate(tokenizer_sbt.word_index)])
#bdd_token_index = dict([(char, i) for i, char in enumerate(tokenizer_bdd.word_index)])

In [23]:
#input_data = np.zeros((len(padded_sbt), max_len_sequence_sbt, len(tokenizer_sbt.word_index)),dtype='float32')
#output_data = np.zeros((len(padded_bdd), max_len_sequence_sbt, len(tokenizer_bdd.word_index)),dtype='float32')


In [24]:
#padded_sbt[0]

In [25]:
'''
for i , (input_text, target_text) in enumerate(zip(padded_sbt,padded_bdd)):
  for t, char in enumerate(padded_sbt[i]):
    print(char)
    input_data[i,t,char] = 1.
  break

   ''' 

'\nfor i , (input_text, target_text) in enumerate(zip(padded_sbt,padded_bdd)):\n  for t, char in enumerate(padded_sbt[i]):\n    print(char)\n    input_data[i,t,char] = 1.\n  break\n\n   '

In [26]:
'''
print(input_data.shape)
print(output_data.shape)
'''

'\nprint(input_data.shape)\nprint(output_data.shape)\n'

# decodage

In [27]:
def sequence_to_sentence(sequence, tokenizer):
  decoded_sequence = []
  for i in sequence:
    for word, index in tokenizer.word_index.items():
      if i == index:
        decoded_sequence.append(word)
        decoded_sequence.append(' ')
  return ''.join(decoded_sequence)


In [28]:
padded_bdd[0].shape

(220,)

In [29]:
print(sequence_to_sentence(padded_sbt[1],tokenizer_sbt))

( methoddeclaration ( modifier public ) modifier public ( modifier static ) modifier static ( simplename fonction 2 ) simplename fonction 2 ( parameter ( primitivetype int ) primitivetype int ( simplename k ) simplename k ) parameter ( parameter ( primitivetype int ) primitivetype int ( simplename m ) simplename m ) parameter ( primitivetype boolean ) primitivetype boolean ( blockstmt ( expressionstmt ( variabledeclarationexpr ( variabledeclarator ( primitivetype int ) primitivetype int ( simplename l ) simplename l ( unaryexpr ( integerliteralexpr 239 ) integerliteralexpr 239 ) unaryexpr ) variabledeclarator ) variabledeclarationexpr ) expressionstmt ( expressionstmt ( variabledeclarationexpr ( variabledeclarator ( primitivetype int ) primitivetype int ( simplename z ) simplename z ( integerliteralexpr 276 ) integerliteralexpr 276 ) variabledeclarator ) variabledeclarationexpr ) expressionstmt ( expressionstmt ( variabledeclarationexpr ( variabledeclarator ( primitivetype boolean ) pr

In [30]:
print(sequence_to_sentence(padded_bdd[1],tokenizer_bdd))

feature: check the fonction 2 # check the result to true scenario: bdd fonction 2 1 given k = -363 and m = 399 when i call fonction 2 then the result should be true 


# Creation du Modele

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, TimeDistributed, LSTM, Bidirectional, Embedding, RepeatVector
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [32]:
model = Sequential()
model.add(Embedding(len(tokenizer_sbt.word_index)+1,256))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
#model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(tokenizer_bdd.word_index)+1, activation='softmax'))
model.compile(loss=sparse_categorical_crossentropy, optimizer='adam',metrics=['accuracy'])

In [33]:

padded_bdd.shape

(1000, 220)

In [34]:
model.fit(padded_sbt,padded_bdd,batch_size = 32, epochs=30,shuffle=False)

Epoch 1/30
32/32 [==============================] - 134s 4s/step - loss: 3.9510 - accuracy: 0.7374
Epoch 2/30
32/32 [==============================] - 129s 4s/step - loss: 0.7245 - accuracy: 0.8482
Epoch 3/30
32/32 [==============================] - 129s 4s/step - loss: 0.6581 - accuracy: 0.8531
Epoch 4/30
32/32 [==============================] - 130s 4s/step - loss: 0.6324 - accuracy: 0.8548
Epoch 5/30
32/32 [==============================] - 130s 4s/step - loss: 0.6102 - accuracy: 0.8591
Epoch 6/30
32/32 [==============================] - 131s 4s/step - loss: 0.5678 - accuracy: 0.8717
Epoch 7/30
32/32 [==============================] - 130s 4s/step - loss: 0.4985 - accuracy: 0.8911
Epoch 8/30
32/32 [==============================] - 130s 4s/step - loss: 0.4278 - accuracy: 0.9157
Epoch 9/30
32/32 [==============================] - 130s 4s/step - loss: 0.3679 - accuracy: 0.9286
Epoch 10/30
32/32 [==============================] - 129s 4s/step - loss: 0.3202 - accuracy: 0.9419
Epoch 11/

KeyboardInterrupt: ignored

In [ ]:
model.save('v1.h5')

In [ ]:
prediction = model.predict(padded_sbt[5])


In [ ]:
prediction.shape

In [ ]:
prediction = np.squeeze(prediction, axis=1)


In [ ]:
prediction[0]

In [ ]:
test = np.argmax(prediction[0])

In [ ]:
test

In [ ]:
prediction.shape

In [ ]:
  pred_sequence = []
  for i in range(len(prediction)):
      pred_sequence.append(np.argmax(prediction[i]))

In [ ]:
pred_sequence

# Test

In [ ]:
for i in range(10):
  true_translation = df['bdd'][i]
  #original_text = lines_en[i]
  #print(original_text)
  print(' true bdd : \n',true_translation)

  #new_tmp = np.expand_dims(tmp_x[i],axis=0)
  #print(new_tmp.shape)
  prediction = model.predict(padded_sbt[i])
  prediction = np.squeeze(prediction, axis=1)
  pred_sequence = []
  for i in range(len(prediction)):
      pred_sequence.append(np.argmax(prediction[i]))
  true_pred = sequence_to_sentence(pred_sequence, tokenizer_bdd)
  print('prediction bdd: \n',true_pred)
  print("\n")

